In [ ]:
# jetbot_control.py (JetBot - WebSocket Server + JetBot Control + Image Send)

import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time
import random

# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# --- JetBot ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리: 단순화된 동작 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")

    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)  # 기본 속도 0.4
            duration = parameters.get("duration", 1.0) # 기본 시간 1초.
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7) # 기본값
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left") if parameters else "left"

            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0}) # turn_left 호출
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0}) # turn_right 호출

            await handle_command("forward", {"speed": 0.4, "duration":1.0}) # move_forward 호출

        elif command == "stop":
            robot.stop()
        elif command == "rotate":  # 회전 명령 처리, 각도와 속도 받기
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3) #기본 속도
                duration = abs(angle) / 45.0 * 0.7  # 45도 회전에 0.7초 기준
                if angle > 0:
                    robot.right(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")


        elif command == "custom_command": # 사용자 지정
            if parameters and "prompt" in parameters:
                 logger.info(f"Received Custom command prompt: {parameters['prompt']}") # 로그
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none":
            pass
        else:
            logger.warning(f"Unknown command: {command}")

    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")


# --- 웹소켓 서버 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        image_task = asyncio.ensure_future(send_images(websocket))

        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling command: {e}")
    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")
    finally:
        image_task.cancel()
        try:
            await image_task
        except asyncio.CancelledError:
            pass

async def start_websocket_server():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started (port: {WEBSOCKET_PORT})")
    await server.wait_closed()

# --- 이미지 전송 함수 ---
async def send_images(websocket):
    while True:
        try:
            image = camera.value
            if image is None:
                logger.warning("Camera image is None")
                await asyncio.sleep(0.1)
                continue
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
            #await asyncio.sleep(0.1) # 이미지 전송 간격 주석처리
        except websockets.exceptions.ConnectionClosedOK:
            logger.info("WebSocket connection closed (image send)")
            break
        except Exception as e:
            logger.exception(f"Error sending image: {e}")
            break

async def main():
    await start_websocket_server()

import sys # sys 임포트

if __name__ == "__main__":
    # Python 버전 확인 및 실행
    if sys.version_info >= (3, 7):  # Python 3.7 이상
        asyncio.run(main())
    else:  # Python 3.6 이하
        loop = asyncio.get_event_loop()
        try:
            loop.run_until_complete(main())
        except RuntimeError as e:
            if "Event loop is already running" not in str(e):
                raise
            else:
                logger.info("Event loop already running.  Continuing...")
        finally:
            try:
                loop.close()
            except RuntimeError as e:
                if "Cannot close a running event loop" not in str(e):
                    raise
                else:
                    logger.info("Event loop already running.  Continuing...")

In [1]:
# jetbot_control.py (JetBot - WebSocket Server + JetBot Control + Image Send)

import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time
import random

# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# --- JetBot ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")

    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)  # 기본 속도 0.4
            duration = parameters.get("duration", 1.0) # 기본 시간 1초.
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7) # 기본값
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left") if parameters else "left"

            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0}) # turn_left 호출
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0}) # turn_right 호출

            await handle_command("forward", {"speed": 0.4, "duration":1.0}) # move_forward 호출

        elif command == "stop":
            robot.stop()
        elif command == "rotate":
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3) # 회전 속도
                duration = abs(angle) / 45.0 * 0.7  # 45도 회전에 0.7초 기준
                if angle > 0:
                    robot.right(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")
        elif command == "custom_command":  # 사용자 지정 명령
            if parameters and "prompt" in parameters:
                 logger.info(f"Received Custom command prompt: {parameters['prompt']}") # 로그
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none":
            pass
        else:
            logger.warning(f"Unknown command: {command}")

    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 웹소켓 서버 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        # 이미지 전송 태스크를 별도로 생성하지 않음
        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling command: {e}")

    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")

async def start_websocket_server():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started (port: {WEBSOCKET_PORT})")
    await server.wait_closed()

# --- 이미지 전송 함수 (수정) ---
async def send_image(websocket):
    """
    현재 카메라 이미지를 가져와서 웹소켓을 통해 전송합니다.
    """
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
    except Exception as e:
        logger.error(f"Error sending image: {e}")
        # 연결이 끊어졌을 때, 더 이상 이미지를 보내지 않음.
        # raise  # 오류를 다시 발생시켜서 websocket_handler에서 처리하도록 함

async def main():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started on port {WEBSOCKET_PORT}")
    await server.wait_closed()

import sys

if __name__ == "__main__":
    if sys.version_info >= (3, 7):
        asyncio.run(main())
    else:
        loop = asyncio.get_event_loop()
        try:
            loop.run_until_complete(main())
        except RuntimeError as e:
            if "Event loop is already running" not in str(e):
                raise
        finally:
            loop.close()

DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Setting PWM frequency to 1600 Hz
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Estimated pre-scale: 2
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Final pre-scale: 3
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2

RuntimeError: Cannot close a running event loop

INFO:__main__:WebSocket server started on port 8766


In [ ]:
# jetbot_control.py (JetBot - WebSocket Server + JetBot Control + Image Send)

import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time
import random

# --- 로깅 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 300
CAMERA_HEIGHT = 300

# --- JetBot ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 ---
async def handle_command(command: str, parameters: dict = None):
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")

    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)  # 기본 속도 0.4
            duration = parameters.get("duration", 1.0) # 기본 시간 1초.
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7) # 기본값
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left") if parameters else "left"

            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0}) # turn_left 호출
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0}) # turn_right 호출

            await handle_command("forward", {"speed": 0.4, "duration":1.0}) # move_forward 호출

        elif command == "stop":
            robot.stop()
        elif command == "rotate":
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3) # 회전 속도
                duration = abs(angle) / 45.0 * 0.7  # 45도 회전에 0.7초 기준
                if angle > 0:
                    robot.right(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed) # speed
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")
        elif command == "custom_command":  # 사용자 지정 명령
            if parameters and "prompt" in parameters:
                 logger.info(f"Received Custom command prompt: {parameters['prompt']}") # 로그
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none":
            pass
        else:
            logger.warning(f"Unknown command: {command}")

    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 웹소켓 서버 ---
async def websocket_handler(websocket, path):
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        #명령어 처리 루프
        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling command: {e}")

            # 매 명령 처리 후, 이미지 전송
            await send_image(websocket)


    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")


async def start_websocket_server():
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started (port: {WEBSOCKET_PORT})")
    await server.wait_closed()

# --- 이미지 전송 함수 (수정) ---
async def send_image(websocket):
    """
    현재 카메라 이미지를 가져와서 웹소켓을 통해 전송합니다.
    """
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
    except Exception as e:
        logger.error(f"Error sending image: {e}")
        # 연결이 끊어졌을 때, 더 이상 이미지를 보내지 않음.
        # raise  # 오류를 다시 발생시켜서 websocket_handler에서 처리하도록 함

async def main():
    await start_websocket_server()

import sys

if __name__ == "__main__":
    # Python 버전 확인 및 실행
    if sys.version_info >= (3, 7):  # Python 3.7 이상
        asyncio.run(main())
    else:  # Python 3.6 이하
        loop = asyncio.get_event_loop()
        try:
            loop.run_until_complete(main())
        except RuntimeError as e:
            if "Event loop is already running" not in str(e):
                raise
            else: # 이미 실행중
                logger.info("Event loop already running.  Continuing...")
        finally:
            # loop.close() # 닫기 주석처리
            pass # 아무것도 안함.

In [1]:
# jetbot_control.py (JetBot - WebSocket Server + JetBot Control + Image Send)

import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- 로깅 설정 ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# --- 설정 ---
WEBSOCKET_PORT = 8766  # 웹소켓 포트
CAMERA_WIDTH = 300     # 카메라 너비
CAMERA_HEIGHT = 300    # 카메라 높이

# --- JetBot 객체 초기화 ---
robot = Robot()
camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)

# --- JetBot 명령 처리 함수 ---
async def handle_command(command: str, parameters: dict = None):
    """
    클라이언트로부터 받은 명령을 처리합니다.

    Args:
        command (str): 실행할 명령어.
        parameters (dict, optional): 명령어 실행에 필요한 추가 파라미터. Defaults to None.
    """
    logger.info(f"Handling JetBot command: {command}, Parameters: {parameters}")

    try:
        if command == "forward":
            speed = parameters.get("speed", 0.4)  # 기본 속도
            duration = parameters.get("duration", 1.0)  # 기본 지속 시간
            robot.forward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "backward":
            speed = parameters.get("speed", 0.4)
            duration = parameters.get("duration", 1.0)
            robot.backward(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "left":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.left(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "right":
            speed = parameters.get("speed", 0.3)
            duration = parameters.get("duration", 0.7)
            robot.right(speed)
            await asyncio.sleep(duration)
            robot.stop()
        elif command == "avoid_obstacle":
            direction = parameters.get("direction", "left")
            if direction == "left":
                await handle_command("left", {"speed": 0.5, "duration": 1.0})
            elif direction == "right":
                await handle_command("right", {"speed": 0.5, "duration": 1.0})
            await handle_command("forward", {"speed": 0.4, "duration": 1.0})
        elif command == "stop":
            robot.stop()
        elif command == "rotate":
            if parameters and "angle" in parameters:
                angle = parameters["angle"]
                speed = parameters.get("speed", 0.3)
                duration = abs(angle) / 45.0 * 0.7  # 45도 회전에 0.7초 가정
                if angle > 0:
                    robot.right(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
                else:
                    robot.left(speed)
                    await asyncio.sleep(duration)
                    robot.stop()
            else:
                logger.warning("Rotate command missing 'angle' parameter")
        elif command == "custom_command":  # 사용자 지정 명령 처리 (추가 로직 필요)
            if parameters and "prompt" in parameters:
                logger.info(f"Received Custom command prompt: {parameters['prompt']}")
            else:
                logger.warning("custom_command missing prompt")
        elif command == "none": # 추가: 'none' 커맨드
            pass
        else:
            logger.warning(f"Unknown command: {command}")

    except Exception as e:
        logger.exception(f"Error in handle_command: {e}")

# --- 이미지 전송 함수 ---
async def send_image(websocket):
    """
    현재 카메라 이미지를 가져와서 웹소켓을 통해 전송합니다.
    """
    try:
        image = camera.value
        if image is not None:
            image_data = bgr8_to_jpeg(image)
            image_base64 = base64.b64encode(image_data).decode('utf-8')
            await websocket.send(json.dumps({"image": image_base64}))
    except Exception as e:
        logger.error(f"Error sending image: {e}")
        # raise  # 오류를 다시 발생시켜서 websocket_handler에서 처리 주석처리


# --- 웹소켓 핸들러 ---
async def websocket_handler(websocket, path):
    """
    웹소켓 클라이언트 연결을 처리하고, 메시지를 수신하여 명령을 처리하고, 이미지를 전송합니다.
    """
    logger.info(f"New WebSocket connection: {websocket.remote_address}")
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                logger.debug(f"Received message: {data}")
                if "command" in data:
                    await handle_command(data["command"], data.get("parameters"))
                else:
                    logger.warning("Invalid message format")

                await send_image(websocket)  # 매 메시지 처리 후 이미지 전송

            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
            except Exception as e:
                logger.exception(f"Error handling message: {e}")  # 수정: 좀 더 일반적인 오류 메시지

    except websockets.exceptions.ConnectionClosedOK:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.exception(f"WebSocket error: {e}")


# --- 메인 함수 (asyncio 이벤트 루프 시작) ---
async def main():
    """
    웹소켓 서버를 시작합니다.
    """
    server = await websockets.serve(websocket_handler, "0.0.0.0", WEBSOCKET_PORT)
    logger.info(f"WebSocket server started on port {WEBSOCKET_PORT}")
    await server.wait_closed()

# --- 스크립트 실행 ---
import sys

if __name__ == "__main__":
     if sys.version_info >= (3, 7):
        asyncio.run(main())
     else:
        loop = asyncio.get_event_loop()
        try:
            loop.run_until_complete(main())
        except RuntimeError as e:
            if "Event loop is already running" not in str(e):
                raise
            logger.info("Event loop already running.")
        finally:
            pass # loop close 제거.

DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Setting PWM frequency to 1600 Hz
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Estimated pre-scale: 2
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Final pre-scale: 3
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 from register 0x00
DEBUG:Adafruit_I2

RuntimeError: This event loop is already running